In [1]:
import numpy as np
from random import shuffle
 
#train_input = ['{0:020b}'.format(i) for i in range(2**20)]
train_input = ['{0:020b}'.format(i) for i in range(2**3)]
shuffle(train_input)
train_input = [map(int,i) for i in train_input]
ti  = []
for i in train_input:
    temp_list = []
    for j in i:
            temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = ti


In [2]:
train_output = []
 
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    temp_list = ([0]*21)
    temp_list[count]=1
    train_output.append(temp_list)
    
print(len(train_output))
print(len(train_input))


8
8


In [3]:
#NUM_EXAMPLES = 10000
NUM_EXAMPLES = 4
test_input = train_input[NUM_EXAMPLES:]
test_output = train_output[NUM_EXAMPLES:] #everything beyond 10,000
 
train_input = train_input[:NUM_EXAMPLES]
train_output = train_output[:NUM_EXAMPLES] #till 10,000

In [4]:
import tensorflow as tf

data = tf.placeholder(tf.float32, [None, 20,1])
target = tf.placeholder(tf.float32, [None, 21])

In [5]:
num_hidden = 24
cell = tf.nn.rnn_cell.LSTMCell(num_hidden,state_is_tuple=True)

In [6]:
val, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)

In [7]:
val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)

In [8]:
weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))

In [9]:
prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)

In [10]:
cross_entropy = -tf.reduce_sum(target * tf.log(tf.clip_by_value(prediction,1e-10,1.0)))

In [11]:
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

/opt/ds/lib/python3.4/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [12]:
mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

In [13]:
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [14]:
batch_size = 1000
no_of_batches = int(len(train_input)/batch_size)
epoch = 8
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        sess.run(minimize,{data: inp, target: out})
    print ("Epoch - ",str(i))
#incorrect = sess.run(error,{data: test_input, target: test_output})
#print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))


Epoch -  0
Epoch -  1
Epoch -  2
Epoch -  3
Epoch -  4
Epoch -  5
Epoch -  6
Epoch -  7


In [15]:
val = sess.run(prediction,{data: [[[0]]]})
print(val)
sess.close()

ValueError: Cannot feed value of shape (1, 1, 1) for Tensor 'Placeholder:0', which has shape '(?, 20, 1)'